Group: Austin Wang, Joe Higgins, Lawrence Moore

# Question 1

In [1]:
# Import necessary modules
import cvxpy as cvx
import numpy as np

## Comparing Average Performance of SOCP, SDP, and NLLS

### Helper functions

#### General Helpers

In [2]:
# Used for generating random anchors and sensors whose coordinates are in [-2,2]
def generate_points(num, dim):
    return np.matrix(4 * np.random.random((num, dim)) - 2)

# For generating the sensor-sensor distances and sensor-anchor distances
def generate_ss_distances(sensors):
    return list(map(lambda s1: 
                list(map(lambda s2: np.linalg.norm(s1 - s2), sensors))
            , sensors))

def generate_sa_distances(sensors, anchors):
    return list(map(lambda s: 
                list(map(lambda a: np.linalg.norm(a - s), anchors))
            , sensors))

# For defining the the matrix-matrix dot product
def sum_elem_product(A,B):
    return cvx.sum_entries(cvx.mul_elemwise(A, B))

# For calculating the number of sensors that are correctly located
def sensor_acc(x, true_sensors):
    num_correct = 0
    for i in range(true_sensors.shape[0]):
        if abs(np.linalg.norm(x[i,:]-true_sensors[i,:])) <= 10**-2:
            num_correct += 1
    return num_correct

#### SDP Helpers

In [3]:
#Make first set of constraint matrices (look like identity)
def enforce_id(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    matrices = []
    rhs = []
    for i in range(dim):
        new_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
        new_matrix[i,i] = 1
        matrices.append(new_matrix)
        rhs.append(1)
        
    return (matrices, rhs)

#Make second set of constraint matrices (symmetric holders) 
def enforce_id2(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    matrices = []
    rhs = []
    for i in range(dim):
        for j in range(dim):
            new_matrix = np.identity(dim)
            if(j > i):
                new_matrix[i,j] = 1
                new_matrix[j,i] = 1
                big_matrix = np.zeros((dim+num_sensors, dim+num_sensors))
                big_matrix[0:dim,0:dim] = new_matrix
                matrices.append(big_matrix)
                rhs.append(dim)
                
    return (matrices, rhs)


#Make third set of constraint matrices (anchors to sensors)
def sensor_constraints(sensors):
    dim = np.shape(sensors)[1]
    num_sensors = np.shape(sensors)[0]
    d_ss = generate_ss_distances(sensors)
    matrices = []
    rhs = []
    zero_vec_dim = np.zeros(dim)
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[i] = 1
            zero_vec_num_s[j] = -1
            
            new_vec = np.matrix(np.append(zero_vec_dim, zero_vec_num_s))
            
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            
            matrices.append(new_matrix)
            rhs.append(d_ss[i][j]**2)

    return (matrices, rhs)

#Make fourth set of constraint matrices (sensors to sensors)
def anchor_constraints(sensors, anchors):
    num_sensors = np.shape(sensors)[0]
    num_anchors = np.shape(anchors)[0]
    d_sa = generate_sa_distances(sensors, anchors)
    matrices = []
    rhs = []
    for i in range(num_anchors):
        for j in range(num_sensors):
            zero_vec_num_s = np.zeros(num_sensors)
            zero_vec_num_s[j] = -1

            new_vec = np.append(np.array(anchors[i,:]), np.array(zero_vec_num_s))
            new_vec = np.matrix(new_vec)
            new_matrix = np.dot(np.transpose(new_vec), new_vec)
            matrices.append(new_matrix)
            rhs.append(d_sa[j][i]**2)

    return (matrices, rhs)

#### NLLS Helpers

In [4]:
# Gradient of objective
def grad(sensors, anchors, sens2sens_dist, sens2anc_dist):
    dim = sensors.shape[1]
    sensor_distance_sum = np.zeros((1,dim))
    anchor_distance_sum = np.zeros((1,dim))
    gradient = np.zeros(sensors.shape)

    for i, sensor_i in enumerate(sensors):
        for j, sensor_j in enumerate(sensors):
            if(i != j):
                sensor_distance_sum += (np.linalg.norm(sensor_i - sensor_j)**2 - \
                                        np.linalg.norm(sens2sens_dist[i][j])**2) * \
                                       (sensor_i - sensor_j)

        for k, anchor_k in enumerate(anchors):
            anchor_distance_sum += (np.linalg.norm(anchor_k - sensor_i)**2 - \
                                    np.linalg.norm(sens2anc_dist[i][k])**2) * \
                                   (sensor_i - anchor_k)

        gradient[i,:] = 8*sensor_distance_sum + 4*anchor_distance_sum

    return gradient

#### Individual Method Run Helpers

In [5]:
# Returns the number of correctly located sensors for SOCP
def run_socp(sensors, anchors):
    d_ss = generate_ss_distances(sensors)
    d_sa = generate_sa_distances(sensors, anchors)
    
    dim = sensors.shape[1]
    num_sensors = 10
    num_anchors = anchors.shape[0]
    x = cvx.Variable(num_sensors, dim)
    objective = cvx.Minimize(0)
    
    constraints  = []
    for i in range(num_sensors):
        x_i = x[i, :]
        for j in range(num_anchors):
            constraints.append(cvx.norm(x_i - anchors[j]) <= d_sa[i][j])
        for j in range(num_sensors):
            if i < j:
                constraints.append(cvx.norm(x_i - sensors[j]) <= d_ss[i][j])

    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver = 'MOSEK')
    return sensor_acc(x.value, sensors)

In [6]:
# Returns the number of correctly located sensors for SDP
def run_sdp(sensors, anchors):    
    
    # Make constraints
    A = enforce_id(sensors)
    B = enforce_id2(sensors)
    C = sensor_constraints(sensors)
    D = anchor_constraints(sensors, anchors)
    
    # Parameters
    dim = sensors.shape[1]
    num_sensors = 10
    num_anchors = anchors.shape[0]
    Z = cvx.Semidef(num_sensors + dim)
    
    constraints = []
    for id_constraint, rhs in zip(A[0], A[1]):
        constraints.append(sum_elem_product(id_constraint, Z) == rhs)
    for id_constraint2, rhs in zip(B[0], B[1]):
        constraints.append(sum_elem_product(id_constraint2, Z) == rhs)
    for sensor_constraint, rhs in zip(C[0], C[1]):
        constraints.append(sum_elem_product(sensor_constraint, Z) == rhs)
    for anchor_constraint, rhs in zip(D[0], D[1]):
        constraints.append(sum_elem_product(anchor_constraint, Z) == rhs)
        
    objective = cvx.Minimize(0)
    prob = cvx.Problem(objective, constraints)
    result = prob.solve(solver = 'MOSEK')
    sol = np.transpose(Z[0:dim, dim:dim+num_sensors].value)
    
    return sensor_acc(sol, sensors)

In [7]:
def run_nlls(sensors, anchors):
    check = 1000
    max_iter = 10000
    k = 0

    # Parameters and initial data
    dim = sensors.shape[1]
    num_sensors = 10
    num_anchors = anchors.shape[0]
    d_ss = generate_ss_distances(sensors)
    d_sa = generate_sa_distances(sensors, anchors)
    
    # Initial sensors guess
    
    # RESULTS QUITE SENSITIVE TO INITIAL GUESS AS WELL
    sensors_0 = np.zeros(sensors.shape)
    sensors_k = sensors_0


    # Iteration
    alpha = .0001 # Result quite sensitive to alpha; chose from empirical evidence
    while check > 10**-6 and k < max_iter:
        sensors_k1 = sensors_k - alpha * grad(sensors_k, anchors, d_ss, d_sa)
        check = np.linalg.norm(sensors_k1 - sensors_k)
        sensors_k = sensors_k1
        k = k+1
    return sensor_acc(sensors_k1, sensors)

In [8]:
# Test (should be removed in final notebook)
#sensors = generate_points(10,3)
#anchors = generate_points(4,3)
#run_nlls(sensors,anchors)

# NOTE: NEED TO TRY DIFFERENT ALPHAS AND INITIAL SENSOR GUESSES TO MAKE
# CONVERGE TO PROPER SOLUTION

In [9]:
# Function that takes in a given dimension dim in [1,2,3] and outputs
# a tuple of 3 lists of lists. Each of these three outer lists corresponds
# to the data for SOCP, SDP, and Nonlinear Least Squares. The inner lists each
# correspond to the number of anchors in [2,3,4]. The values in each of these inner
# lists are the average number of sensors out of the 10 that were located correctly
# out of 50 trials
def execute_solvers(dim):
    SOCP_vals = []
    SDP_vals = []
    NLLS_vals = []
    for anc_i in num_anchors:
        socp_sum = 0
        sdp_sum = 0
        nlls_sum = 0
        for j in range(50):
            # Initialize data for a given trial
            sensors = generate_points(10, dim)
            anchors = generate_points(anc_i, dim)
            
            socp_sum += run_socp(sensors, anchors)
            sdp_sum += run_sdp(sensors, anchors)
            nlls_sum += run_nlls(sensors, anchors)
        SOCP_vals.append(socp_sum/50)
        SDP_vals.append(sdp_sum/50)
        NLLS_vals.append(nlls_sum/50)
        print('Finished One Anchor Iteration')

    return (SOCP_vals, SDP_vals, NLLS_vals)


In [10]:
num_anchors = [2,3,4]

In [ ]:
dim1 = execute_solvers(1)

Finished One Anchor Iteration
Finished One Anchor Iteration
Finished One Anchor Iteration


In [ ]:
dim2 = execute_solvers(2)

In [ ]:
dim3 = execute_solvers(3)